<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/DeepFactor_YJ1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install mxnet and gluonts

In [0]:
pip install --upgrade mxnet==1.6 gluonts

     |████████████████████████████████| 68.7MB 65kB/s 
     |████████████████████████████████| 419kB 46.2MB/s 
     |████████████████████████████████| 194kB 47.8MB/s 
     |████████████████████████████████| 7.4MB 45.4MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68017 sha256=910f9518b301934bfba2a01a294000d1fdfb469699d14440fe62e62678a17315
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


## Dataset Loading

In [0]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
#import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import json
#import os

In [0]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
#from gluonts.dataset.util import to_pandas

In [0]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

In [0]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.distribution.gaussian import GaussianOutput
from gluonts.distribution.student_t import StudentTOutput
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

## Experiment of DeepFactor

## electricity

In [0]:
elec_dataset = get_dataset("electricity", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/test/data.json


In [0]:
print (len(elec_dataset.train))
print (len(elec_dataset.test))

321
2247


In [0]:
elec_dataset.metadata

MetaData(freq='1H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [0]:
elec_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')

In [0]:
car = [int(elec_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(elec_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
test_ds_24 = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(elec_dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=elec_dataset.metadata.freq)

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepfactor_trainer=Trainer(
    ctx = 'cpu',
    epochs=100, # (default: 100).
    batch_size=32,# (default: 32).
    num_batches_per_epoch=50, # (default: 50).
    learning_rate=0.001, # (default:  10^−3 ).
    learning_rate_decay_factor=0.5, # (default: 0.5).
    patience = 10, # (default: 10).
    minimum_learning_rate=5e-05, # (default:  5⋅10^−5 ).
    clip_gradient = 10.0, # (default: 10).
    weight_decay=1e-08, #  (default  10^−8 ).
    init='xavier', #  (default: “xavier”).
    hybridize=True, 
    )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=elec_dataset.metadata.freq, # Time series frequency
    prediction_length=elec_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(elec_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:09<00:00,  5.16it/s, epoch=100/100, avg_epoch_loss=7.78e+3]


In [0]:
# Evaluation for all test data
forecast_it, ts_it = make_evaluation_predictions(
    dataset=elec_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(elec_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 41557.06it/s]


{
    "MSE": 25993367.61071011,
    "abs_error": 35274492.36671448,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 2.648145734615828,
    "MAPE": 0.2922686890965108,
    "sMAPE": 0.29726672704196405,
    "OWA": NaN,
    "MSIS": 76.59054661067174,
    "QuantileLoss[0.1]": 27819210.360164642,
    "Coverage[0.1]": 0.26707832665776593,
    "QuantileLoss[0.5]": 35274492.24931861,
    "Coverage[0.5]": 0.39500815902685016,
    "QuantileLoss[0.9]": 39991089.11282539,
    "Coverage[0.9]": 0.5583184987390599,
    "RMSE": 5098.3691128350165,
    "NRMSE": 2.1374370772989733,
    "ND": 0.2742259329460988,
    "wQuantileLoss[0.1]": 0.21626814173627978,
    "wQuantileLoss[0.5]": 0.2742259320334566,
    "wQuantileLoss[0.9]": 0.3108930273889173,
    "mean_wQuantileLoss": 0.2671290337195512,
    "MAE_Coverage": 0.2045838896306186
}


In [0]:
# Evaluation for only 24H prediction test data 
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 7587.33it/s]


{
    "MSE": 11237881.446448408,
    "abs_error": 4224858.509498596,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 2.400687726079926,
    "MAPE": 0.27595824507620137,
    "sMAPE": 0.2968769958977066,
    "OWA": NaN,
    "MSIS": 67.19668753857242,
    "QuantileLoss[0.1]": 3615421.5741348267,
    "Coverage[0.1]": 0.2625908618899275,
    "QuantileLoss[0.5]": 4224858.443302065,
    "Coverage[0.5]": 0.38798026998961566,
    "QuantileLoss[0.9]": 4443340.76124382,
    "Coverage[0.9]": 0.5606178608515056,
    "RMSE": 3352.2949521855035,
    "NRMSE": 1.495517099900887,
    "ND": 0.244649906969038,
    "wQuantileLoss[0.1]": 0.2093590944589791,
    "wQuantileLoss[0.5]": 0.24464990313577942,
    "wQuantileLoss[0.9]": 0.25730161174061356,
    "mean_wQuantileLoss": 0.23710353644512402,
    "MAE_Coverage": 0.20466424368293543
}


## exchange_rate

In [0]:
exchange_rate_dataset = get_dataset("exchange_rate", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/exchange_rate/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/exchange_rate/test/data.json


In [0]:
print (len(exchange_rate_dataset.train))
print (len(exchange_rate_dataset.test))

8
40


In [0]:
exchange_rate_dataset.metadata

MetaData(freq='1B', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=30)

In [0]:
exchange_rate_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')

In [0]:
car = [int(exchange_rate_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(exchange_rate_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=exchange_rate_dataset.metadata.freq, # Time series frequency
    prediction_length=exchange_rate_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(exchange_rate_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:03<00:00, 12.96it/s, epoch=100/100, avg_epoch_loss=-4.95]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=exchange_rate_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(exchange_rate_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 40/40 [00:00<00:00, 907.24it/s]


{
    "MSE": 0.002736776959985339,
    "abs_error": 45.29600842297077,
    "abs_target_sum": 975.9766580164433,
    "abs_target_mean": 0.8133138816803692,
    "seasonal_error": 0.006515919403041841,
    "MASE": 74.44956299622991,
    "MAPE": 0.3323232758775952,
    "sMAPE": 0.17889929541209057,
    "OWA": NaN,
    "MSIS": 2918.558188344827,
    "QuantileLoss[0.1]": 45.52110567092896,
    "Coverage[0.1]": 0.5658333333333333,
    "QuantileLoss[0.5]": 45.296008456498384,
    "Coverage[0.5]": 0.615,
    "QuantileLoss[0.9]": 39.17810098715127,
    "Coverage[0.9]": 0.6541666666666667,
    "RMSE": 0.05231421374717715,
    "NRMSE": 0.06432229293700478,
    "ND": 0.04641095465851564,
    "wQuantileLoss[0.1]": 0.046641592600631665,
    "wQuantileLoss[0.5]": 0.046410954692868524,
    "wQuantileLoss[0.9]": 0.040142456958731074,
    "mean_wQuantileLoss": 0.04439833475074375,
    "MAE_Coverage": 0.27555555555555555
}


## m4_daily

In [0]:
m4_daily_dataset = get_dataset("m4_daily", regenerate=True)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


In [0]:
print (len(m4_daily_dataset.train))
print (len(m4_daily_dataset.test))

4227
4227


In [0]:
m4_daily_dataset.metadata

MetaData(freq='D', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=14)

In [0]:
m4_daily_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')

In [0]:
car = [int(m4_daily_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
m4_daily_dataset.metadata.feat_static_cat

[CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')]

In [0]:
m4_daily_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')

In [0]:
next(iter(m4_daily_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=m4_daily_dataset.metadata.freq, # Time series frequency
    prediction_length=m4_daily_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(m4_daily_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:02<00:00, 22.13it/s, epoch=100/100, avg_epoch_loss=1.88e+4]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_daily_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_daily_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 4227/4227 [00:00<00:00, 68888.39it/s]


{
    "MSE": 1148196273.5156221,
    "abs_error": 1059233491.6230774,
    "abs_target_sum": 383097653.5168457,
    "abs_target_mean": 6473.649895515986,
    "seasonal_error": 52.78075560259988,
    "MASE": 312.1557271938038,
    "MAPE": 2.411727807274379,
    "sMAPE": 0.8847168145180249,
    "OWA": NaN,
    "MSIS": 12439.9404255331,
    "QuantileLoss[0.1]": 1899205754.2938597,
    "Coverage[0.1]": 0.9360742167697453,
    "QuantileLoss[0.5]": 1059233493.0277863,
    "Coverage[0.5]": 0.9404508432187636,
    "QuantileLoss[0.9]": 216400867.7806152,
    "Coverage[0.9]": 0.9436952921693873,
    "RMSE": 33885.0449832315,
    "NRMSE": 5.234302986743566,
    "ND": 2.764917722411736,
    "wQuantileLoss[0.1]": 4.95749774726915,
    "wQuantileLoss[0.5]": 2.764917726078448,
    "wQuantileLoss[0.9]": 0.5648712953317516,
    "mean_wQuantileLoss": 2.7624289228931165,
    "MAE_Coverage": 0.4400734507192987
}


## m4_hourly

In [0]:
m4_hourly_dataset = get_dataset("m4_hourly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/test/data.json


In [0]:
print (len(m4_hourly_dataset.train))
print (len(m4_hourly_dataset.test))

414
414


In [0]:
m4_hourly_dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='414')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=48)

In [0]:
m4_hourly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='414')

In [0]:
car = [int(m4_hourly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_hourly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=m4_hourly_dataset.metadata.freq, # Time series frequency
    prediction_length=m4_hourly_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(m4_hourly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:02<00:00, 23.10it/s, epoch=100/100, avg_epoch_loss=2.69e+5]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_hourly_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_hourly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 414/414 [00:00<00:00, 10187.70it/s]


{
    "MSE": 40983661.08745066,
    "abs_error": 20809180.209091187,
    "abs_target_sum": 145558863.59960938,
    "abs_target_mean": 7324.822041043147,
    "seasonal_error": 336.9046924038302,
    "MASE": 11.039941629714814,
    "MAPE": 0.8920382287944805,
    "sMAPE": 0.33377137651036604,
    "OWA": NaN,
    "MSIS": 163.94602963485542,
    "QuantileLoss[0.1]": 23649332.642423436,
    "Coverage[0.1]": 0.2813003220611915,
    "QuantileLoss[0.5]": 20809180.075209618,
    "Coverage[0.5]": 0.5357789855072465,
    "QuantileLoss[0.9]": 17659643.281104468,
    "Coverage[0.9]": 0.7369162640901772,
    "RMSE": 6401.848255578279,
    "NRMSE": 0.8739936915472933,
    "ND": 0.14296058442948045,
    "wQuantileLoss[0.1]": 0.16247263861221092,
    "wQuantileLoss[0.5]": 0.14296058350970434,
    "wQuantileLoss[0.9]": 0.12132303622320846,
    "mean_wQuantileLoss": 0.14225208611504123,
    "MAE_Coverage": 0.1267210144927536
}


## m4_monthly

In [0]:
m4_monthly_dataset = get_dataset("m4_monthly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/test/data.json


In [0]:
print (len(m4_monthly_dataset.train))
print (len(m4_monthly_dataset.test))

48000
48000


In [0]:
m4_monthly_dataset.metadata

MetaData(freq='M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=18)

In [0]:
m4_monthly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')

In [0]:
car = [int(m4_monthly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_monthly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=m4_monthly_dataset.metadata.freq, # Time series frequency
    prediction_length=m4_monthly_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(m4_monthly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:01<00:00, 41.02it/s, epoch=100/100, avg_epoch_loss=1.48e+4]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_monthly_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_monthly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 48000/48000 [00:00<00:00, 325805.98it/s]


{
    "MSE": 32115441897.68937,
    "abs_error": 51222334701.30725,
    "abs_target_sum": 4157157295.381714,
    "abs_target_mean": 4811.5246474325495,
    "seasonal_error": 616.9975625355751,
    "MASE": 166.2121247462581,
    "MAPE": 11.856855778301268,
    "sMAPE": 1.4860916522621186,
    "OWA": NaN,
    "MSIS": 6641.4769399733905,
    "QuantileLoss[0.1]": 88926061869.27962,
    "Coverage[0.1]": 0.2678310185185138,
    "QuantileLoss[0.5]": 51222334749.17082,
    "Coverage[0.5]": 0.2688171296296251,
    "QuantileLoss[0.9]": 13478121088.436924,
    "Coverage[0.9]": 0.26986689814814335,
    "RMSE": 179207.81762436975,
    "NRMSE": 37.24553665541251,
    "ND": 12.321481017379684,
    "wQuantileLoss[0.1]": 21.391074609582304,
    "wQuantileLoss[0.5]": 12.321481028893217,
    "wQuantileLoss[0.9]": 3.242148451637876,
    "mean_wQuantileLoss": 12.318234696704465,
    "MAE_Coverage": 0.34304899691358176
}


## m4_quarterly

In [0]:
m4_quarterly_dataset = get_dataset("m4_quarterly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_quarterly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_quarterly/test/data.json


In [0]:
print (len(m4_quarterly_dataset.train))
print (len(m4_quarterly_dataset.test))

24000
24000


In [0]:
m4_quarterly_dataset.metadata

MetaData(freq='3M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='24000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=8)

In [0]:
m4_quarterly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='24000')

In [0]:
car = [int(m4_quarterly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_quarterly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=m4_quarterly_dataset.metadata.freq, # Time series frequency
    prediction_length=m4_quarterly_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(m4_quarterly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:00<00:00, 71.18it/s, epoch=100/100, avg_epoch_loss=3.55e+4]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_quarterly_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_quarterly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 24000/24000 [00:00<00:00, 257294.27it/s]


{
    "MSE": 81002322.32597734,
    "abs_error": 1098683036.5536804,
    "abs_target_sum": 1147074070.9031982,
    "abs_target_mean": 5974.344119287491,
    "seasonal_error": 473.4332698179725,
    "MASE": 19.83702600151264,
    "MAPE": 0.9552013469974565,
    "sMAPE": 1.516506232904913,
    "OWA": NaN,
    "MSIS": 787.1556528289225,
    "QuantileLoss[0.1]": 754048331.036651,
    "Coverage[0.1]": 0.13260416666666666,
    "QuantileLoss[0.5]": 1098683036.6063194,
    "Coverage[0.5]": 0.13318229166666667,
    "QuantileLoss[0.9]": 1435098741.8476853,
    "Coverage[0.9]": 0.13377083333333334,
    "RMSE": 9000.129017185105,
    "NRMSE": 1.5064631091686218,
    "ND": 0.9578135051806942,
    "wQuantileLoss[0.1]": 0.6573667299818908,
    "wQuantileLoss[0.5]": 0.9578135052265839,
    "wQuantileLoss[0.9]": 1.2510950933776217,
    "mean_wQuantileLoss": 0.9554251095286989,
    "MAE_Coverage": 0.38855034722222226
}


## m4_weekly

In [0]:
m4_weekly_dataset = get_dataset("m4_weekly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_weekly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_weekly/test/data.json


In [0]:
print (len(m4_weekly_dataset.train))
print (len(m4_weekly_dataset.test))

359
359


In [0]:
m4_weekly_dataset.metadata

MetaData(freq='W', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='359')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=13)

In [0]:
m4_weekly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='359')

In [0]:
car = [int(m4_weekly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_weekly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=m4_weekly_dataset.metadata.freq, # Time series frequency
    prediction_length=m4_weekly_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(m4_weekly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:01<00:00, 41.10it/s, epoch=100/100, avg_epoch_loss=1.49e+3]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_weekly_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_weekly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 359/359 [00:00<00:00, 6401.56it/s]


{
    "MSE": 3092125.678778238,
    "abs_error": 5337996.949234009,
    "abs_target_sum": 25616889.502441406,
    "abs_target_mean": 5488.941397566194,
    "seasonal_error": 216.54813922503882,
    "MASE": 19.148520440241946,
    "MAPE": 0.2117046438547443,
    "sMAPE": 0.23647832069685804,
    "OWA": NaN,
    "MSIS": 734.0715815119916,
    "QuantileLoss[0.1]": 1482328.4895713804,
    "Coverage[0.1]": 0.12384829655024661,
    "QuantileLoss[0.5]": 5337996.920166016,
    "Coverage[0.5]": 0.15084636811656327,
    "QuantileLoss[0.9]": 8908309.51903839,
    "Coverage[0.9]": 0.19112920505678205,
    "RMSE": 1758.4441073796568,
    "NRMSE": 0.32036124637063057,
    "ND": 0.20837802921878057,
    "wQuantileLoss[0.1]": 0.057865280225770886,
    "wQuantileLoss[0.5]": 0.20837802808406072,
    "wQuantileLoss[0.9]": 0.34775141291800427,
    "mean_wQuantileLoss": 0.20466490707594528,
    "MAE_Coverage": 0.3606242411256338
}


## m4_yearly

In [0]:
m4_yearly_dataset = get_dataset("m4_yearly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_yearly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_yearly/test/data.json


In [0]:
print (len(m4_yearly_dataset.train))
print (len(m4_yearly_dataset.test))

23000
23000


In [0]:
m4_yearly_dataset.metadata

MetaData(freq='12M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='23000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=6)

In [0]:
m4_yearly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='23000')

In [0]:
car = [int(m4_yearly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_yearly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=m4_yearly_dataset.metadata.freq, # Time series frequency
    prediction_length=m4_yearly_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(m4_yearly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:00<00:00, 80.58it/s, epoch=100/100, avg_epoch_loss=7.73e+5]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_yearly_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_yearly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 23000/23000 [00:00<00:00, 247913.88it/s]


{
    "MSE": 2958111464.9457064,
    "abs_error": 3132234639.657791,
    "abs_target_sum": 860724679.6655273,
    "abs_target_mean": 6237.135359895143,
    "seasonal_error": 288.79964609563865,
    "MASE": 92.63026527263264,
    "MAPE": 3.1088665698625637,
    "sMAPE": 1.6513679605655567,
    "OWA": NaN,
    "MSIS": 3697.2276293340437,
    "QuantileLoss[0.1]": 4835222257.744239,
    "Coverage[0.1]": 0.22566666666666663,
    "QuantileLoss[0.5]": 3132234640.508995,
    "Coverage[0.5]": 0.2258985507246376,
    "QuantileLoss[0.9]": 1423850317.3690596,
    "Coverage[0.9]": 0.22618115942028993,
    "RMSE": 54388.52328337024,
    "NRMSE": 8.720112703195303,
    "ND": 3.63906683943925,
    "wQuantileLoss[0.1]": 5.617617772529979,
    "wQuantileLoss[0.5]": 3.639066840428188,
    "wQuantileLoss[0.9]": 1.6542459522856479,
    "mean_wQuantileLoss": 3.6369768550812718,
    "MAE_Coverage": 0.3578623188405797
}


## traffic

In [0]:
traffic_dataset = get_dataset("traffic", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/traffic/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/traffic/test/data.json


In [0]:
print (len(traffic_dataset.train))
print (len(traffic_dataset.test))

862
6034


In [0]:
traffic_dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='862')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [0]:
traffic_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='862')

In [0]:
car = [int(traffic_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(traffic_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=traffic_dataset.metadata.freq, # Time series frequency
    prediction_length=traffic_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(traffic_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:07<00:00,  7.13it/s, epoch=100/100, avg_epoch_loss=-2.65]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=traffic_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(traffic_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 6034/6034 [00:00<00:00, 58968.44it/s]


{
    "MSE": 0.002561205598955793,
    "abs_error": 4733.029822394252,
    "abs_target_sum": 8672.5710073933,
    "abs_target_mean": 0.05988682885450042,
    "seasonal_error": 0.015220711169889612,
    "MASE": 2.295675025370343,
    "MAPE": 2.2527044089286816,
    "sMAPE": 0.6060083968169826,
    "OWA": NaN,
    "MSIS": 58.1787454762361,
    "QuantileLoss[0.1]": 3125.494466629658,
    "Coverage[0.1]": 0.3311443486907536,
    "QuantileLoss[0.5]": 4733.029828500352,
    "Coverage[0.5]": 0.5085211578831068,
    "QuantileLoss[0.9]": 4200.5266754503355,
    "Coverage[0.9]": 0.6809123301292679,
    "RMSE": 0.05060835503111905,
    "NRMSE": 0.8450665363176247,
    "ND": 0.5457470245397104,
    "wQuantileLoss[0.1]": 0.3603884550458218,
    "wQuantileLoss[0.5]": 0.5457470252437807,
    "wQuantileLoss[0.9]": 0.48434618429407134,
    "mean_wQuantileLoss": 0.4634938881945579,
    "MAE_Coverage": 0.15291772548153085
}


## electricity2

In [0]:
elec_dataset = get_dataset("electricity", regenerate=False)

In [0]:
print (len(elec_dataset.train))
print (len(elec_dataset.test))

321
2247


In [0]:
elec_dataset.metadata

MetaData(freq='1H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [0]:
elec_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')

In [0]:
car = [int(elec_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(elec_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
test_ds_24 = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(elec_dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=elec_dataset.metadata.freq)

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deep_factor.html
DF_estimator = DeepFactorEstimator(
    freq=elec_dataset.metadata.freq, # Time series frequency
    prediction_length=elec_dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # (default: 50).
    num_layers_global=1, # (default: 1).
    num_factors=10, # (default: 10).
    num_hidden_local=5, # (default: 5).
    num_layers_local=1, # (default: 1).
    cell_type='lstm', # (default: ‘lstm’).
    trainer = deepfactor_trainer, # (default: Trainer()).
    context_length=None, # (default: None, in which case context_length = prediction_length).
    num_parallel_samples=100, # (default: 100).
    cardinality=car, # (default: list([1]).
    embedding_dimension=10, # (default: 10).
    distr_output=StudentTOutput(), # (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(elec_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:09<00:00,  5.04it/s, epoch=100/100, avg_epoch_loss=1.01e+4]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=elec_dataset.test, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(elec_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 27507.20it/s]


{
    "MSE": 38011133.90213028,
    "abs_error": 54565493.10239029,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 3.9024437813883703,
    "MAPE": 0.43935678380373255,
    "sMAPE": 0.3480442307626229,
    "OWA": NaN,
    "MSIS": 126.06130128789589,
    "QuantileLoss[0.1]": 87105837.37931862,
    "Coverage[0.1]": 0.5967030114226384,
    "QuantileLoss[0.5]": 54565493.2295739,
    "Coverage[0.5]": 0.7506675567423224,
    "QuantileLoss[0.9]": 19288360.186621856,
    "Coverage[0.9]": 0.8502076843198325,
    "RMSE": 6165.317015541884,
    "NRMSE": 2.584743648541697,
    "ND": 0.42419528244682714,
    "wQuantileLoss[0.1]": 0.6771657908515966,
    "wQuantileLoss[0.5]": 0.42419528343555984,
    "wQuantileLoss[0.9]": 0.14994882172047616,
    "mean_wQuantileLoss": 0.41710329866921086,
    "MAE_Coverage": 0.26572096128170947
}


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DF_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 4287.89it/s]


{
    "MSE": 38731749.65495565,
    "abs_error": 8171835.9200668335,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 4.143609634307464,
    "MAPE": 0.45505713616111676,
    "sMAPE": 0.37235982025449244,
    "OWA": NaN,
    "MSIS": 134.97233582829844,
    "QuantileLoss[0.1]": 13748536.40445061,
    "Coverage[0.1]": 0.638759086188993,
    "QuantileLoss[0.5]": 8171835.93080458,
    "Coverage[0.5]": 0.783489096573209,
    "QuantileLoss[0.9]": 2194503.7526630396,
    "Coverage[0.9]": 0.869807892004154,
    "RMSE": 6223.483723362314,
    "NRMSE": 2.7764043623832504,
    "ND": 0.4732084857080486,
    "wQuantileLoss[0.1]": 0.7961398339724427,
    "wQuantileLoss[0.5]": 0.47320848632984186,
    "wQuantileLoss[0.9]": 0.1270776613524827,
    "mean_wQuantileLoss": 0.46547532721825574,
    "MAE_Coverage": 0.28414676358601604
}
